In [1]:
import transformers
from datasets import load_dataset, load_metric

In [4]:
news = load_dataset("csv", data_files="./data/news_collection.csv", split="train")
# DatasetDict({
#    train: Dataset({
#        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
#        num_rows: 192368
#    })
# })

Using custom data configuration default-cf0f959afad0821a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/Users/rakson/miniforge3/envs/pytorch-m1/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /Users/rakson/.cache/huggingface/datasets/csv/default-cf0f959afad0821a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [5]:
news

Dataset({
    features: ['title', 'desc', 'image', 'url', 'source', 'date'],
    num_rows: 142663
})